### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [2]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712337 entries, 0 to 712336
Data columns (total 23 columns):
user_id      712337 non-null int64
movie_id     712337 non-null int64
rating       712337 non-null int64
timestamp    712337 non-null int64
date         712337 non-null object
month_1      712337 non-null int64
month_2      712337 non-null int64
month_3      712337 non-null int64
month_4      712337 non-null int64
month_5      712337 non-null int64
month_6      712337 non-null int64
month_7      712337 non-null int64
month_8      712337 non-null int64
month_9      712337 non-null int64
month_10     712337 non-null int64
month_11     712337 non-null int64
month_12     712337 non-null int64
year_2013    712337 non-null int64
year_2014    712337 non-null int64
year_2015    712337 non-null int64
year_2016    712337 non-null int64
year_2017    712337 non-null int64
year_2018    712337 non-null int64
dtypes: int64(22), object(1)
memory usage: 125.0+ MB


In [5]:
reviews.groupby("movie_id").rating.mean()[:10]

movie_id
8       5.000000
10     10.000000
12     10.000000
25      8.000000
91      6.000000
417     8.368421
439     6.750000
443     8.000000
628     4.500000
833     3.000000
Name: rating, dtype: float64

In [6]:
reviews.groupby("movie_id").rating.count()[:10]

movie_id
8       1
10      1
12      1
25      1
91      1
417    19
439     4
443     1
628     2
833     1
Name: rating, dtype: int64

In [7]:
reviews.groupby("movie_id").timestamp.max()[:10]

movie_id
8      1396981211
10     1412878553
12     1439248579
25     1488189899
91     1385233195
417    1530047326
439    1529809149
443    1477256912
628    1385729728
833    1385738847
Name: timestamp, dtype: int64

In [14]:
tmp = pd.concat([movies[["movie_id", "movie"]].set_index("movie_id"),
          reviews.groupby("movie_id").rating.mean(),
          reviews.groupby("movie_id").rating.count(),
          reviews.groupby("movie_id").timestamp.max()
          ], axis=1).head(10)

In [17]:
tmp.columns = ["movie", "rating_mean", "review_count", "timestamp_max"]
tmp.sort_values(by=["rating_mean", "review_count", "timestamp_max"], ascending=False)

,movie,rating_mean,review_count,timestamp_max
movie_id,,,,
12,The Arrival of a Train (1896),10.000000,1,1439248579
10,La sortie des usines Lumière (1895),10.000000,1,1412878553
417,Le voyage dans la lune (1902),8.368421,19,1530047326
25,The Oxford and Cambridge University Boat Race ...,8.000000,1,1488189899
443,"Hiawatha, the Messiah of the Ojibway (1903)",8.000000,1,1477256912
439,The Great Train Robbery (1903),6.750000,4,1529809149
91,Le manoir du diable (1896),6.000000,1,1385233195
8,Edison Kinetoscopic Record of a Sneeze (1894),5.000000,1,1396981211
628,The Adventures of Dollie (1908),4.500000,2,1385729728


In [18]:
tmp[tmp.review_count >= 5]

,movie,rating_mean,review_count,timestamp_max
movie_id,,,,
417,Le voyage dans la lune (1902),8.368421,19,1530047326


In [23]:
tmp.movie.tolist()[:10]

['Edison Kinetoscopic Record of a Sneeze (1894)',
 'La sortie des usines Lumière (1895)',
 'The Arrival of a Train (1896)',
 'The Oxford and Cambridge University Boat Race (1895)',
 'Le manoir du diable (1896)',
 'Le voyage dans la lune (1902)',
 'The Great Train Robbery (1903)',
 'Hiawatha, the Messiah of the Ojibway (1903)',
 'The Adventures of Dollie (1908)',
 'The Country Doctor (1909)']

In [29]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    mov_rev = pd.concat([movies[["movie_id", "movie"]].set_index("movie_id"),
          reviews.groupby("movie_id").rating.mean(),
          reviews.groupby("movie_id").rating.count(),
          reviews.groupby("movie_id").timestamp.max()
          ], axis=1)
    mov_rev.columns = ["movie", "rating_mean", "review_count", "timestamp_max"]
    mov_rev = mov_rev.sort_values(by=["rating_mean", "review_count", "timestamp_max"], ascending=False)
    
    top_movies = mov_rev[mov_rev.review_count >= 5].movie.tolist()[:n_top]
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [30]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1, 20)# Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5)# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35)# Your solution list here



In [ ]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

In [28]:
ranked_movies.head()

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.0,28,2016-01-08 00:44:43
5688932,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,14,2018-06-17 01:44:48
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,10,2015-05-10 22:56:01
2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,10.0,6,2016-01-23 00:30:44


In [31]:
# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


In [32]:
t.popular_recommendations('1', 20, ranked_movies), recs_20_for_1

(['MSG 2 the Messenger (2015)',
  'Avengers: Age of Ultron Parody (2015)',
  'Sorry to Bother You (2018)',
  'Selam (2013)',
  "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
  'Crawl Bitch Crawl (2012)',
  'Make Like a Dog (2015)',
  'Pandorica (2016)',
  'Third Contact (2011)',
  'Romeo Juliet (2009)',
  'Be Somebody (2016)',
  'Birlesen Gonuller (2014)',
  'Agnelli (2017)',
  'Sátántangó (1994)',
  'Shijie (2004)',
  'Foster (2011)',
  'CM101MMXI Fundamentals (2013)',
  'Akahige (1965)',
  'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
  'Körkarlen (1921)'],
 ['MSG 2 the Messenger (2015)',
  'Avengers: Age of Ultron Parody (2015)',
  'Sorry to Bother You (2018)',
  'Selam (2013)',
  "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
  'Crawl Bitch Crawl (2012)',
  'Make Like a Dog (2015)',
  'Pandorica (2016)',
  'Third Contact (2011)',
  'Romeo Juliet (2009)',
  'Be Somebody (2016)',
  'Birlesen Gonuller (2014)',
  'Agnelli (20

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [36]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=[], genres=[]):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a dataframe with movies that are sorted by highest avg rating, more reviews, 
                    then time, and must have more than 4 ratings
    years - a list of years to filter results on
    genres - a list of genres to filter results on
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff

    # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
    
    top_movies = ranked_movies.movie.tolist()[:n_top]
    
    return top_movies # a list of the n_top movies as recommended

In [34]:
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']

In [37]:
popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

['Ayla: The Daughter of War (2017)',
 'I Believe in Miracles (2015)',
 'The Farthest (2017)',
 'Sado (2015)',
 'Hatred (2016)',
 'Kincsem (2017)',
 'Nise - O Coração da Loucura (2015)',
 'LA 92 (2017)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Dunkirk (2017)',
 'Taeksi Woonjunsa (2017)',
 'The Battleship Island (2017)',
 'Darkest Hour (2017)',
 'Best of Enemies (2015)',
 'The Ghazi Attack (2017)']